## Extension to Lab1: Training for multivariate data

In [1]:
import cv2
import scipy.stats as ss
from mpmath import *
import numpy as np
import matplotlib.pyplot as plt
import math
%xmode plain

Exception reporting mode: Plain


In [2]:
def read_img(img_name='train.jpg', mask_name='mask.jpg'):
    
    img = cv2.imread(img_name, 1)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    mask = cv2.imread(mask_name,1)
    mask = cv2.cvtColor(mask, cv2.COLOR_RGB2GRAY)
    
    return np.array(img).astype(float), np.array(mask).astype(float)

In [3]:
img, mask = read_img()

In [4]:
def get_pixels(img, mask, num_pixels):
    
    if num_pixels:
        ncoin = np.sum(mask > 200 )
        ndesk = np.sum(mask <= 200 )
        return ncoin, ndesk
    else:
        coin = img[(mask > 200).nonzero()]
        desk = img[(mask <= 200).nonzero()]
        return coin, desk

## Multivariable MLE

In [5]:
def ML(img, mask, percentage, c_or_d):
    
    coin_pxl, desk_pxl = get_pixels(img, mask, False)
    coin_pxl_v = coin_pxl[:,2]
    desk_pxl_v = desk_pxl[:,2]
    coin_pxl_s = coin_pxl[:,1]
    desk_pxl_s = desk_pxl[:,1]
    coin_pxl_h = coin_pxl[:,0]
    desk_pxl_h = desk_pxl[:,0]
    
    
    # proportion of data to access
    num_elements_coin = int(np.floor(percentage*len(coin_pxl)))
    num_elements_desk = int(np.floor(percentage*len(desk_pxl)))
    
    # sigma calculation functions
    calc_mu =  lambda x: np.mean(x)
    calc_sigma = lambda x, mu: np.mean((x-mu)**2)
    
    if  c_or_d:
        # coin paramaters HSV
        coin_mu_h = calc_mu(coin_pxl_h[0:num_elements_coin])
        coin_sig_h = calc_sigma(coin_pxl_h[0:num_elements_coin], coin_mu_h)
        coin_mu_s = calc_mu(coin_pxl_s[0:num_elements_coin])
        coin_sig_s = calc_sigma(coin_pxl_s[0:num_elements_coin], coin_mu_s)
        coin_mu_v = calc_mu(coin_pxl_v[0:num_elements_coin])
        coin_sig_v = calc_sigma(coin_pxl_v[0:num_elements_coin], coin_mu_v)
        
        print("h mu coin {:.6f} sigma desk {:.6f} ____".format(coin_mu_h, coin_sig_h))
        print("s mu coin {:.6f} sigma desk {:.6f} ____".format(coin_mu_s, coin_sig_s))
        print("v mu coin {:.6f} sigma desk {:.6f} ____".format(coin_mu_v, coin_sig_v))
        
        return coin_mu_h, coin_sig_h, coin_mu_s, coin_sig_s, coin_mu_v, coin_sig_v
    
    else:
        # coin parameters 
        desk_mu_h = calc_mu(desk_pxl_h[0:num_elements_coin])
        desk_sig_h = calc_sigma(desk_pxl_h[0:num_elements_coin], desk_mu_h)
        desk_mu_s = calc_mu(desk_pxl_s[0:num_elements_coin])
        desk_sig_s = calc_sigma(desk_pxl_s[0:num_elements_coin], desk_mu_s)
        desk_mu_v = calc_mu(desk_pxl_v[0:num_elements_coin])
        desk_sig_v = calc_sigma(desk_pxl_v[0:num_elements_coin], desk_mu_v)
        print("h mu desk {:.6f} sigma desk {:.6f} ____".format(desk_mu_h, desk_sig_h))
        print("s mu desk {:.6f} sigma desk {:.6f} ____".format(desk_mu_s, desk_sig_s))
        print("v mu desk {:.6f} sigma desk {:.6f} ____".format(desk_mu_v, desk_sig_v))
        
        return desk_mu_h, desk_sig_h, desk_mu_s, desk_sig_s, desk_mu_v, desk_sig_v
     
def get_prob(pixels_h, pixels_s, pixels_v, mu_h, var_h, mu_s, var_s, mu_v, var_v):
    
    p1_h = ss.norm.pdf(pixels_h, mu_h, np.sqrt(var_h))
    p1_s = ss.norm.pdf(pixels_s, mu_s, np.sqrt(var_s))
    p1_v = ss.norm.pdf(pixels_v, mu_v, np.sqrt(var_v))
    
    return p1_h, p1_s, p1_v 

def test_model(img, mask, c_mu_h, c_var_h,c_mu_s, c_var_s, c_mu_v, c_var_v, eps):
    
    # getting probabilities for h
    img_reshape = img[:,:, 0].flatten()
    mask_reshape = mask.flatten()
    chunk_size =int(np.floor(eps*len(img_reshape)))
    pxl_h = img_reshape[0:chunk_size]
    
    # getting probabilities for s
    img_reshape = img[:,:, 1].flatten()
    mask_reshape = mask.flatten()
    chunk_size =int(np.floor(eps*len(img_reshape)))
    pxl_s = img_reshape[0:chunk_size]
    
    # getting probabilities for v
    img_reshape = img[:,:, 2].flatten()
    mask_reshape = mask.flatten()
    chunk_size =int(np.floor(eps*len(img_reshape)))
    pxl_v = img_reshape[0:chunk_size]
    
    p1_h, p1_s, p1_v = get_prob(pxl_h, pxl_s, pxl_v, c_mu_h, c_var_h,c_mu_s, c_var_s, c_mu_v, c_var_v)
    
    return p1_h, p1_s, p1_v
    
def check_acc(mask, probs, eps=1):
    dims = mask.shape
    mask_reshape = mask.flatten()
    chunk_size =int(np.floor(eps*len(mask_reshape)))
    mask_pxl = mask_reshape[0:chunk_size]
    probs[ probs > 0.5] = 255
    probs[ probs <= 0.5] = 0
    mask_pxl[ mask_pxl > 200] = 255
    mask_pxl[ mask_pxl <= 200] = 0
    
    cv2.imshow('output', probs.reshape(dims))
    cv2.waitKey()
    cv2.destroyAllWindows()
    
    return np.sum(x == mask_pxl)/chunk_size

In [8]:
coin_mu_h, coin_sig_h, coin_mu_s, coin_sig_s, coin_mu_v, coin_sig_v = ML(img, mask, 1, True)
desk_mu_h, desk_sig_h, desk_mu_s, desk_sig_s, desk_mu_v, desk_sig_v = ML(img, mask, 1, False)
p1_h, p1_s, p1_v = test_model(img, mask, coin_mu_h, coin_sig_h, coin_mu_s, coin_sig_s, coin_mu_v, coin_sig_v, eps=1)
p2_h, p2_s, p2_v = test_model(img, mask, desk_mu_h, desk_sig_h, desk_mu_s, desk_sig_s, desk_mu_v, desk_sig_v, eps=1)

h mu coin 28.696981 sigma desk 263.037778 ____
s mu coin 20.364947 sigma desk 210.434602 ____
v mu coin 203.028564 sigma desk 1155.544580 ____
h mu desk 69.990357 sigma desk 2611.327998 ____
s mu desk 7.259648 sigma desk 50.755070 ____
v mu desk 121.970236 sigma desk 115.982336 ____


#### Need to add the prior

In [9]:
x = (p1_h*p1_s*p1_v)/((p1_h*p1_s*p1_v) + (p2_h*p2_s*p2_v))
check_acc(mask, x, 1)

0.8319156901041667

In [ ]:
img1, mask1 = read_img(img_name='val1.jpg', mask_name='val_mask1.png')
p1_h, p1_s, p1_v = test_model(img1, mask1, coin_mu_h, coin_sig_h, coin_mu_s, coin_sig_s, coin_mu_v, coin_sig_v, eps=0.5)
p2_h, p2_s, p2_v = test_model(img1, mask1, desk_mu_h, desk_sig_h, desk_mu_s, desk_sig_s, desk_mu_v, desk_sig_v, eps=0.5)
x = (p1_h*p1_s*p1_v)/((p1_h*p1_s*p1_v) + (p2_h*p2_s*p2_v))
check_acc(mask1, x, 0.5)
print("validation image 1: ", check_acc(mask1, x, 0.5))

###################################
img2, mask2 = read_img(img_name='val2.jpg', mask_name='val_mask2.png')
p1_h, p1_s, p1_v = test_model(img2, mask2, coin_mu_h, coin_sig_h, coin_mu_s, coin_sig_s, coin_mu_v, coin_sig_v, eps=0.5)
p2_h, p2_s, p2_v = test_model(img2, mask2, desk_mu_h, desk_sig_h, desk_mu_s, desk_sig_s, desk_mu_v, desk_sig_v, eps=0.5)
x = (p1_h*p1_s*p1_v)/((p1_h*p1_s*p1_v) + (p2_h*p2_s*p2_v))
check_acc(mask2, x, 0.5)
print("validation image 2: ", check_acc(mask2, x, 0.5))

## Multivariable MAPE

In [ ]:
def MAPE(img, mask, percentage, c_or_d):
    
    coin_pxl, desk_pxl = get_pixels(img, mask, False)
    coin_pxl_v = coin_pxl[:,2]
    desk_pxl_v = desk_pxl[:,2]
    coin_pxl_s = coin_pxl[:,1]
    desk_pxl_s = desk_pxl[:,1]
    coin_pxl_h = coin_pxl[:,0]
    desk_pxl_h = desk_pxl[:,0]
    
    
    # proportion of data to access
    num_elements_coin = int(np.floor(percentage*len(coin_pxl)))
    num_elements_desk = int(np.floor(percentage*len(desk_pxl)))
    
    # sigma calculation functions
    calc_mu = lambda x: (np.sum(x))/(len(x)+1)
    calc_sigma = lambda x, mu: (np.sum((x-mu)**2) + 2 + (-mu)**2)/(len(x) + 5)
    
    if  c_or_d:
        # coin paramaters HSV
        coin_mu_h = calc_mu(coin_pxl_h[0:num_elements_coin])
        coin_sig_h = calc_sigma(coin_pxl_h[0:num_elements_coin], coin_mu_h)
        coin_mu_s = calc_mu(coin_pxl_s[0:num_elements_coin])
        coin_sig_s = calc_sigma(coin_pxl_s[0:num_elements_coin], coin_mu_s)
        coin_mu_v = calc_mu(coin_pxl_v[0:num_elements_coin])
        coin_sig_v = calc_sigma(coin_pxl_v[0:num_elements_coin], coin_mu_v)
        
        print("h mu coin {:.6f} sigma desk {:.6f} ____".format(coin_mu_h, coin_sig_h))
        print("s mu coin {:.6f} sigma desk {:.6f} ____".format(coin_mu_s, coin_sig_s))
        print("v mu coin {:.6f} sigma desk {:.6f} ____".format(coin_mu_v, coin_sig_v))
        
        return coin_mu_h, coin_sig_h, coin_mu_s, coin_sig_s, coin_mu_v, coin_sig_v
    
    else:
        # coin parameters 
        desk_mu_h = calc_mu(desk_pxl_h[0:num_elements_coin])
        desk_sig_h = calc_sigma(desk_pxl_h[0:num_elements_coin], desk_mu_h)
        desk_mu_s = calc_mu(desk_pxl_s[0:num_elements_coin])
        desk_sig_s = calc_sigma(desk_pxl_s[0:num_elements_coin], desk_mu_s)
        desk_mu_v = calc_mu(desk_pxl_v[0:num_elements_coin])
        desk_sig_v = calc_sigma(desk_pxl_v[0:num_elements_coin], desk_mu_v)
        print("h mu desk {:.6f} sigma desk {:.6f} ____".format(desk_mu_h, desk_sig_h))
        print("s mu desk {:.6f} sigma desk {:.6f} ____".format(desk_mu_s, desk_sig_s))
        print("v mu desk {:.6f} sigma desk {:.6f} ____".format(desk_mu_v, desk_sig_v))
        
        return desk_mu_h, desk_sig_h, desk_mu_s, desk_sig_s, desk_mu_v, desk_sig_v

In [ ]:
coin_mu_h, coin_sig_h, coin_mu_s, coin_sig_s, coin_mu_v, coin_sig_v = MAPE(img, mask, 0.5, True)
desk_mu_h, desk_sig_h, desk_mu_s, desk_sig_s, desk_mu_v, desk_sig_v = MAPE(img, mask, 0.5, False)
p1_h, p1_s, p1_v = test_model(img, mask, coin_mu_h, coin_sig_h, coin_mu_s, coin_sig_s, coin_mu_v, coin_sig_v, eps=0.5)
p2_h, p2_s, p2_v = test_model(img, mask, desk_mu_h, desk_sig_h, desk_mu_s, desk_sig_s, desk_mu_v, desk_sig_v, eps=0.5)

In [ ]:
x = (p1_h*p1_s*p1_v)/((p1_h*p1_s*p1_v) + (p2_h*p2_s*p2_v))
check_acc(mask, x, 0.5)

In [ ]:
img1, mask1 = read_img(img_name='val1.jpg', mask_name='val_mask1.png')
p1_h, p1_s, p1_v = test_model(img1, mask1, coin_mu_h, coin_sig_h, coin_mu_s, coin_sig_s, coin_mu_v, coin_sig_v, eps=0.5)
p2_h, p2_s, p2_v = test_model(img1, mask1, desk_mu_h, desk_sig_h, desk_mu_s, desk_sig_s, desk_mu_v, desk_sig_v, eps=0.5)
x = (p1_h*p1_s*p1_v)/((p1_h*p1_s*p1_v) + (p2_h*p2_s*p2_v))
check_acc(mask1, x, 0.5)
print("validation image 1: ", check_acc(mask1, x, 0.5))

###################################
img2, mask2 = read_img(img_name='val2.jpg', mask_name='val_mask2.png')
p1_h, p1_s, p1_v = test_model(img2, mask2, coin_mu_h, coin_sig_h, coin_mu_s, coin_sig_s, coin_mu_v, coin_sig_v, eps=0.5)
p2_h, p2_s, p2_v = test_model(img2, mask2, desk_mu_h, desk_sig_h, desk_mu_s, desk_sig_s, desk_mu_v, desk_sig_v, eps=0.5)
x = (p1_h*p1_s*p1_v)/((p1_h*p1_s*p1_v) + (p2_h*p2_s*p2_v))
check_acc(mask2, x, 0.5)
print("validation image 2: ", check_acc(mask2, x, 0.5))